In [34]:
#https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
%load_ext autoreload
%autoreload 2
from datetime import datetime
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv
from Kafka import KafkaReader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
from pkgutil import get_data

def get_dataframe() -> pd.DataFrame:
    kafka = KafkaReader()
    df = pd.DataFrame(columns=['datetime','temp','city'])
    messages = kafka.retrieve()
    kafka.consumer.close()
    maxKey = pd.to_datetime(pd.DataFrame(messages.keys())[0]).max().strftime('%d.%m.%Y %H:%M')
    for f in messages[maxKey]['cities']:
            city = f['city']
            for w in f['weather']:
                dt = w['dt']
                temp = w['temp']
                df.loc[len(df), df.columns] = dt, temp, city
    return df

def load_locations() -> pd.DataFrame:
    df = pd.read_json('locations.json').T
    return df

def get_map_data() -> pd.DataFrame():
    df = pd.merge(left=get_dataframe(), right=load_locations(), left_on='city', right_index=True)
    df['temp'] = pd.to_numeric(df['temp'])
    #df['datetime'] = pd.to_datetime(df['datetime'])
    return df

def create_map() -> None:
    forecasts = get_dataframe()
    print(forecasts.head())

    # set all makers to the same size
    marker_max = 20
    marker_size = [marker_max]*forecasts.shape[0]
    
    # your code
    
    fig.show(renderer='colab')  # set renderer='colab' only when running in jupyter notebook

In [51]:
import plotly.express as px

load_dotenv()
token = os.getenv('MAPBOXKEY')
px.set_mapbox_access_token(token)

df = get_map_data()

fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color="temp", width=800, height=600,
                  color_continuous_scale=px.colors.cyclical.IceFire, range_color=(-10, 40), animation_frame='datetime', animation_group='city')
fig.update_traces(marker={'size': 20})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig["layout"].pop("updatemenus")
fig.show()
